# General overview

-----------------------------------------------------------------------------------------------------------------
The following Jupyter notebook was created in order to automate the process of generating and visualizing forecasts for the macroeconomic indicators using historical time series retrieved using World Bank's Indicators API. Forecast will be generated using ARIMA and SARIMA models. 

# Import libraries and set display options

In [15]:
import json
import pandas as pd
from retrieval_funcs import create_world_bank_api_url_string, retrieve_url_content
from preprocessing_funcs import convert_bytes_to_unicode, extract_dates_and_values_from_json


# Set pandas data display options
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Script parameters  

Please provide the country and economic indicator codes for which you would like to generate the forecast

In [2]:
COUNTRY_CODE = 'afg'
INDICATOR_CODE = 'NY.GDP.MKTP.CN'

# Retrieve and preprocess data 

In [3]:
# Create query string used to retrieve the data using World Bank's Indicators API
query_string = create_world_bank_api_url_string(COUNTRY_CODE, INDICATOR_CODE)

In [4]:
# Retrieve data 
data = retrieve_url_content(query_string)

In [5]:
# Convert retrieved data represented as bytes object to JSON
json_string = convert_bytes_to_unicode(data)
json = json.loads(json_string)

In [6]:
# Extract dates and values from JSON
dates, values = extract_dates_and_values_from_json(json)

In [18]:
# Create DataFrame object using dates and values
time_series_df = pd.DataFrame.from_dict({"Date": dates, "Value": values})

# Change index to Date column
time_series_df.set_index('Date', inplace=True)

In [19]:
# Display first 15 rows of data 
time_series_df.head(15)

,Value
Date,
2019,nan
2018,1400685916120.100
2017,1375505093900.000
2016,1314198682300.000
2015,1222756495400.000
2014,1176469485800.000
2013,1138809791300.000
2012,1028794220100.000
2011,850347275400.000
